In [11]:
from pcntoolkit.util.hbr_utils import *
import scipy

import os
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import pcntoolkit as ptk
from itertools import product
from functools import reduce
from pcntoolkit.model.SHASH import SHASHb, SHASH, SHASHo
from pcntoolkit.util.utils import scaler
from scipy.stats import gaussian_kde
import pymc3 as pm
import scipy.special as spp
import arviz as av
from scipy.stats import skew, kurtosis
import seaborn as sns
import time
sns.set(style='darkgrid')

In [12]:
projdir = '/project_cephfs/3022017.02/projects/stijdboe/make_results/10_folds_results/'
data_dir = '/project_cephfs/3022017.02/projects/stijdboe/Data'
folds_dir = os.path.join(data_dir,'10_folds')
python_path = '/project_cephfs/3022017.02/projects/stijdboe/stijn2/bin/python3.10'
normative_path = '/project_cephfs/3022017.02/projects/stijdboe/PCNtoolkit/pcntoolkit/normative.py'

In [13]:
# For each fold
for i_fold in range(10):
    
    # Load the train and test data
    fold_data_dir = f'/project_cephfs/3022017.02/projects/stijdboe/Data/10_folds_sexcov/fold_{i_fold}'
    with open(os.path.join(fold_data_dir, 'X_train.pkl'),'rb') as file:
              X_train = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Y_train.pkl'),'rb') as file:
              Y_train = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Z_train.pkl'),'rb') as file:
              Z_train = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'X_test.pkl'),'rb') as file:
              X_test = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Y_test.pkl'),'rb') as file:
              Y_test = pickle.load(file)
    with open(os.path.join(fold_data_dir, 'Z_test.pkl'),'rb') as file:
              Z_test = pickle.load(file)

    # For each feature
    features = ['Right-Cerebellum-White-Matter',
                'EstimatedTotalIntraCranialVol',
                'Right-Lateral-Ventricle',
                'WM-hypointensities',
                'rh_S_interm_prim-Jensen_thickness', 
                'Brain-Stem']
    for i_f, feature in enumerate(features):    
        
        # Load that particular data
        this_X_train = X_train.to_numpy()
        this_Y_train = Y_train[feature].to_numpy()
        this_X_test = X_test.to_numpy()
        this_Y_test = Y_test[feature].to_numpy()
        this_Z_test = Z_test.to_numpy()
        
        # Apply in- and outscale to train and test (use util.scaler)
        inscaler = scaler('standardize')
        outscaler = scaler('standardize')
        inscaler.fit(this_X_train)
        outscaler.fit(this_Y_train)
        this_X_test = inscaler.transform(this_X_test)
        this_Y_test = outscaler.transform(this_Y_test)
        
        # For each model type:
        for model_type in ['SHASHb_1','SHASHb_2','Normal']:
            # Load the model
            fold_dir = f'fold_{i_fold}_{model_type}'
            batch_dir = f'batch_{i_f+1}'
            nm_name = f'NM_0_0_estimate1sample.pkl'
            model_path = os.path.join(projdir, fold_dir, batch_dir, 'Models', nm_name)
            if os.path.exists(model_path):
                with open(model_path, 'rb') as file:
                    norm_hbr_model = pickle.load(file)
                # Find the MCMC z-scores
                X_test_transformed = norm_hbr_model.hbr.transform_X(this_X_test)
#                 bspline_transform(this_X_test,norm_hbr_model.hbr.bsp)
                print(f"Computing MAPs for {nm_name}")
                start = time.time()
                map_path = os.path.join(projdir, f"MAPS/MAP_fold{i_fold}_{i_f}_{model_type}.pkl")
                with open(map_path, 'rb') as file:
                    MAP = pickle.load(file)
                print(this_Y_test.shape)
                MCMC_z_scores = get_single_zscores(np.squeeze(X_test_transformed), this_Y_test[:,None], this_Z_test, norm_hbr_model, MAP)[:,0]
                end = time.time()
                print("Found MAP_zscores in ", end-start)
#                 n = np.random.randn(*MCMC_z_scores.shape)
#                 plt.scatter(np.sort(n), np.sort(MCMC_z_scores), alpha = 0.2)
#                 plt.title(f'fold {i_fold}, {feature} {model_type}')
#                 plt.show()
#                 plt.close()
                # Save the MAP
                mcmc_zscoresdir = os.path.join(projdir, fold_dir, batch_dir, 'MAP_zscores')
                if not os.path.exists(mcmc_zscoresdir):
                    os.mkdir(mcmc_zscoresdir)
                with open(os.path.join(mcmc_zscoresdir, f'MAP_zscores_{i_f}_{model_type}_fold{i_fold}.pkl'),'wb') as file:
                    pickle.dump(MCMC_z_scores, file)



Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.05398130416870117
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.0670771598815918
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.015859127044677734
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.057785749435424805
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.058733224868774414
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.016304969787597656
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.10305476188659668
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.05876564979553223
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.01547551155090332
Computing MAPs for NM_0_0_estimate1sample.pkl
(5767,)
Found MAP_zscores in  0.05545473098754883
Computing MAPs for NM_0_0_estimate1sa